In [1]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/tehdog/data/dev/2019/pic2kcal-cv-praktikum:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [2]:
%cd ..

/tf/notebooks


In [3]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece
!pip install tqdm

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0707 18:02:29.588741 139647363516160 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [5]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from math import ceil

data_dir = Path("data")

In [6]:
# jq '.[]' fddb_data_v3.json > fddb_data_v3.jsonl
# jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' fddb_data_v3.jsonl | jq -s > fddb_data_v3_withimg.json
with open(str(data_dir / "fddb_data_v4_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    _out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [7]:
def normalize_ingredient(ing: str):
    ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing)
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

from extract_ingredients.util import normalize_out_ingredient
    

In [8]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
print(len(_in_names))
in_names.sort()
len(in_names)

258944


21444

In [9]:
print(len(_out_names))
out_names = list({ning for ing in _out_names for ning in normalize_out_ingredient(ing)})
out_names.sort()
len(out_names)

122977


108086

In [10]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [11]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [12]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [13]:
def get_match(search: np.array, out_vecs, limit=30):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(limit, it, key=itemgetter(1))
    return res_list

In [14]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [15]:
out_vecs[0][1].shape

(512,)

In [16]:
in_names[0], out_names[0]

('8 Kräuter', '"DIE FEINE" Geflügel-Fleischwurst, würzi...')

In [20]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [21]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

Chiliflocken aus der Mühle -> ('Kirschstreusel', 0.7449405)
Rosmarin . frischer -> ('Rosmarin, frisch', 0.9431499)
Mehl nach Bedarf -> ('Zopfmehl', 0.80514395)
Zartbitterschokolade 70 % Kakaoanteil -> ('Bitterschokolade 70 % Kakaogehalt', 0.89710546)
Frischkäse bis 1 % Fett absolut -> ('Weichkäse, 13% Fett absolut', 0.8758312)
Vollkornnudeln Dinkelvollkornspiralspätzle -> ('Dinkelvollkorn Bauernspätzle', 0.90983033)
Piri-Piri aus dem Glas -> ('Piri-Piri, feurig scharf', 0.74664783)
Hirschfleisch aus der Keule -> ('Schabefleisch vom Rind', 0.8933994)
Kräuter für die Crème fraiche -> ('Crème fraiche mit Kräuter', 0.8813894)
Brotaufstrich Paprika Peperoni -> ('Paprika&Peperoni Brotaufstrich', 0.9281652)
Rettiche und Kohlrabi -> ('Kohlrabi-Gemüse', 0.8378093)
Kürbisfleisch zu Mus gekocht und abgetropft -> ('Kürbiskerne, geröstet und gesalzen', 0.8433009)
Reh - Knochen aus der Keule -> ('Rinder-Gulasch aus der Keule', 0.77878106)
Chorizo -> ('Chorizo', 0.9999998)
Spätzle aus dem Kühlregal -

In [22]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

[('Kartoffeln', 1.0),
 ('Kartoffelstock', 0.8959534),
 ('Kartoffeln roh', 0.8905239),
 ('Kartoffeln gebacken', 0.8904908),
 ('Kartoffelscheiben', 0.8901619),
 ('Kartoffeln, ganz', 0.88885343),
 ('Kartoffel', 0.88085693),
 ('Kartoffelnudeln', 0.8792579),
 ('Gekochte Kartoffeln', 0.8718474),
 ('geschälte Kartoffeln', 0.8708629),
 ('Kartoffel Sticks', 0.8684819),
 ('Kartoffeln mehlig', 0.86703193),
 ('Kartoffelklöße', 0.8620798),
 ('Kartoffelpürree', 0.86069757),
 ('Kartoffelpüree', 0.8605306),
 ('Kartoffeln rot', 0.8586046),
 ('Kartoffeltopf', 0.85467523),
 ('Kartoffelspalten', 0.8534164),
 ('Kartoffeln festkochend', 0.85249126),
 ('Kartoffelteig', 0.8495877),
 ('Kartoffeln in Scheiben', 0.8480123),
 ('Kartoffeln (fertig)', 0.8420578),
 ('Kartoffelcreme', 0.8384942),
 ('Kartoffelmehl', 0.8381945),
 ('Kartoffelbüsch', 0.8379458),
 ('Kartoffeln -  ganz', 0.83605176),
 ('Kartoffelbrei', 0.83593315),
 ('Kartoffel-Pürree', 0.8321384),
 ('Kartoffel-Püree', 0.8320216),
 ('Bratkartoffeln', 0.829

In [23]:
z = list(get_sentence_vectors(["Schalotten"]))[0]
get_match(z, out_vecs, limit=30)

[('Schalotten', 1.0000001),
 ('Sülze', 0.89274013),
 ('Ringlotten', 0.8788649),
 ('Schaschlik', 0.8767235),
 ('Leberkse', 0.87103057),
 ('Schwartenmagen', 0.86798453),
 ('Maiskölbchen', 0.86279553),
 ('Grützwust', 0.8598486),
 ('Maisecke', 0.85870075),
 ('Käsknacker', 0.858381),
 ('Hhnerfilet', 0.8569063),
 ('Korneck', 0.85655665),
 ('Körnereck', 0.8554502),
 ('Landschinken', 0.85422593),
 ('Rohschinken', 0.8509237),
 ('Karreespeck', 0.85052764),
 ('Wafer', 0.84980637),
 ('Kräcker', 0.8489746),
 ('Radieschen', 0.84836495),
 ('Rehgulasch', 0.8480122),
 ('Kareespeck', 0.8477373),
 ('Rohesser', 0.846316),
 ('Körnerbursche', 0.8454909),
 ('Mairübchen', 0.8453922),
 ('Rettich', 0.8451376),
 ('Jausenspeck', 0.8450292),
 ('Kaisersemmeln', 0.8447572),
 ('Stelline', 0.84436196),
 ('Palatschinken', 0.8443409),
 ('Mohnstuten', 0.84411025)]

In [24]:
z = list(get_sentence_vectors([normalize_ingredient("Salz")]))[0]
get_match(z, out_vecs, limit=30)

[('Salz', 1.0),
 ('Salzgurken', 0.9089097),
 ('Gut Salzig', 0.90342623),
 ('Salzbrezeln', 0.8880923),
 ('Salzgurken, salzig', 0.8686831),
 ('Salzmandeln', 0.8676863),
 ('Salzbrezeln, Salzig', 0.8605131),
 ('Salzbrezel', 0.8582727),
 ('Salz-Mandeln', 0.85328484),
 ('salzbrezel', 0.84165347),
 ('Salz, Jodsalz', 0.8251066),
 ('Salzbutter', 0.81924367),
 ('Salzdillgurken', 0.8158475),
 ('Jod Salz', 0.81424797),
 ('Salz flutes, Salz', 0.8119249),
 ('Blutdruck Salz', 0.8090307),
 ('Salz-Stangen', 0.80218107),
 ('smorbar gesalzen', 0.7952043),
 ('Chips Salz', 0.79345155),
 ('Salzstangerl', 0.7885661),
 ('Salz Kräcker', 0.7803569),
 ('Corn, salz', 0.7801466),
 ('Salz Stangen', 0.7790837),
 ('Salzige Heringe', 0.7762423),
 ('Salzmandeln, geröstet gesalzen', 0.76485217),
 ('Streichzart, gesalzen', 0.76328605),
 ('Saltletts', 0.7625023),
 ('Salz-Schneidebohnen', 0.76154387),
 ('Salzstangen', 0.7579998),
 ('Salz-Cashews', 0.7565831)]

In [25]:
z = list(get_sentence_vectors([normalize_ingredient("Möhre(n)")]))[0]
get_match(z, out_vecs, limit=30)

[('Möhren', 0.9999998),
 ('Möhrchen', 0.9716229),
 ('Möhrenaufstrich', 0.91976905),
 ('Möhreneintopf', 0.9090408),
 ('Möhreneintopf ', 0.9090408),
 ('Karotten', 0.90865445),
 ('Möhrengemüse', 0.9066272),
 ('Möhren-Eintopf', 0.8989031),
 ('Möhren Eintopf', 0.8960131),
 ('möhreneintopf', 0.8826865),
 ('Möhrenbrötchen', 0.8769465),
 ('Karotteneintopf', 0.8718132),
 ('Karotten / Möhren', 0.86903),
 ('Möhrensaft', 0.86823654),
 ('Karottenscheiben', 0.86586285),
 ('Babymöhren', 0.8603344),
 ('Karottenbrot', 0.86014485),
 ('baby möhren', 0.85710764),
 ('junge Möhrchen', 0.8510489),
 ('Karottensaft', 0.8468522),
 ('Karottenciabatta', 0.84540284),
 ('Möhren-Eiweiß-Brot', 0.83676785),
 ('Karottenwürfel', 0.8357495),
 ('Karotten gebraten', 0.83498985),
 ('Gemüsebrei', 0.8288224),
 ('Karotten-Salat', 0.8278241),
 ('Möhrchen-Liebe', 0.8260028),
 ('Blätterteig', 0.824074),
 ('Gurkentopf', 0.82374495),
 ('Erbsen & Möhren', 0.82353055)]

In [26]:
# i thought multiprocessing is impossible in jupyter notebook?
# better not question it
from multiprocessing import Pool

def jsonable(e):
    return [(a,float(b)) for a, b in e]

def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(8) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = jsonable(match)

In [27]:

all_ings_orig = {ing: all_ings[normalize_ingredient(ing)] for ing in _in_names}

In [28]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)